In [1]:
%%bash
rm -r Pubyc-Cluster-Overhead

In [2]:
import os
os.mkdir('./Pubyc-Cluster-Overhead')

In [3]:
%%file ./Pubyc-Cluster-Overhead/PubycTaskManager.py
from __future__ import print_function

from multiprocessing.managers import BaseManager
from multiprocessing import JoinableQueue, Queue

class TaskManager(BaseManager):
    pass

if __name__ == '__main__':
    from sys import argv, exit
    if len(argv) != 2:
        print('usage:', argv[0], b'socket_nr')
        exit(0)
    master_socket = int(argv[1])
    task_queue = JoinableQueue()
    result_queue = Queue()
    TaskManager.register('get_job_queue', 
                         callable = lambda:task_queue)
    TaskManager.register('get_result_queue', 
                         callable = lambda:result_queue)
    m = TaskManager(address = ('', master_socket), 
                    authkey = b'secret')
    print('starting queue server, socket', master_socket)
    m.get_server().serve_forever()

Writing ./Pubyc-Cluster-Overhead/PubycTaskManager.py


In [4]:
%%file ./Pubyc-Cluster-Overhead/PubycWorker.py
from PubycTaskManager import TaskManager
from multiprocessing import cpu_count, Process

def worker_single_planet(job_queue, result_queue):
    while True:
        args = job_queue.get()
        result_queue.put(((0, 0, 0, 0, 0, 0), 0))
        job_queue.task_done()
        
def start_workers(m):
    job_queue, result_queue = m.get_job_queue(), m.get_result_queue()
    nr_of_processes = cpu_count()
    processes = [
            Process(target = worker_single_planet, args = (job_queue, result_queue))
            for i in range(nr_of_processes)
    ]
    
    for p in processes:
        p.start()
    return nr_of_processes


if __name__ == '__main__':
    from sys import argv, exit
    if len(argv) < 3:
        print('usage:', argv[0], 'server_IP server_socket')
        exit(0)
    server_ip = argv[1]
    server_socket = int(argv[2])
    TaskManager.register('get_job_queue')
    TaskManager.register('get_result_queue')
    m = TaskManager(address=(server_ip, server_socket), authkey = b'secret')
    m.connect()
    nr_of_processes = start_workers(m)
    print(nr_of_processes, 'workers started')

Writing ./Pubyc-Cluster-Overhead/PubycWorker.py


In [5]:
%%file ./Pubyc-Cluster-Overhead/PubycMaster.py

from multiprocessing import cpu_count
from PubycTaskManager import TaskManager
import numpy as np
import time

def single_step_arguments(partsize, dt, position, speed, masse, l_in): 
    
    planet_number = len(position)-1
    parts = planet_number // partsize
    upper_planet_index = 0
    lower_planet_index = 0
    
    while (upper_planet_index - planet_number) < 0:
        
        if upper_planet_index + partsize < planet_number:
            upper_planet_index += partsize
        else:
            upper_planet_index = planet_number

        l_in.append((dt, position, speed, masse, lower_planet_index, upper_planet_index))  
        lower_planet_index = upper_planet_index + 1 
    
    return l_in

def update_list(result_tuple, position, speed):
        position[result_tuple[1]] = (result_tuple[0][0], result_tuple[0][1], result_tuple[0][2])
        speed[result_tuple[1]] = (result_tuple[0][3], result_tuple[0][4], result_tuple[0][5]) 

def single_step(job_queue, result_queue, partsize, dt, position, speed, masse):
    arguments_list = []
    result_list = []    
    
    t1 = time.time()
    arguments_list = single_step_arguments(partsize, dt, position, speed, masse, arguments_list)
    t2 = time.time()
    print('finished create argument_list with time: ', (t2-t1) * 1000, 'ms')

    
    t1 = time.time()
    for parameter_set in arguments_list:
        #print(parameter_set)
        job_queue.put(parameter_set)
    t2 = time.time()
    print('finished jop_queue with time:            ', (t2-t1) * 1000, 'ms')
    
    
    #while result_queue.empty():
    #    time.sleep(0.05)
    
    t1 = time.time()    
    while not result_queue.empty():
        update_list(result_queue.get(), position, speed)
    t2 = time.time()
    print('finished result_queue with time:         ', (t2-t1) * 1000, 'ms')
        
        
    t1 = time.time()
    job_queue.join()
    t2 = time.time()
    print('finished join with time:                 ', (t2-t1) * 1000, 'ms\n')

        
        
if __name__ == '__main__':
    from sys import argv, exit
    if len(argv) != 8:
        print('usage:', argv[0], 'server_IP server_socket number_of_trys number_of_parts worker_count')
        exit(0)
    server_ip = argv[1]
    server_socket = int(argv[2])
    args_dt = argv[3]
    partsize = argv[4]
    position = argv[5]
    speed = argv[6]
    masse = argv[7]
    
    TaskManager.register('get_job_queue')
    TaskManager.register('get_result_queue')
    taskmanager = TaskManager(address=(server_ip, server_socket), authkey = b'secret')
    taskmanager.connect()

    t1 = time.time()
    result = single_step(taskmanager, partsize, args_dt, position, speed, masse)
    t2 = time.time()
    print(' result: ', result)
    print(' time:   ', t2-t1, ' s\n')

Writing ./Pubyc-Cluster-Overhead/PubycMaster.py


## TimeIt

In [6]:
%load_ext autoreload

In [21]:
from random import random

number_of_bodys = 10000

python_position = []
python_speed = []
python_mass = []

for i in range(number_of_bodys):
    
    python_position.append((random() * 10 ** 12, random() * 10 ** 12, random()* 10 ** 12))
    python_speed.append((random()* 10 ** 5, random()* 10 ** 5, random()* 10 ** 5))
    python_mass.append(random()* 10 ** 30)

In [24]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append("./Pubyc-Cluster-Overhead")

from PubycMaster import single_step
from PubycTaskManager import TaskManager
import numpy as np

position = np.array(python_position, dtype=np.float64)
speed = np.array(python_speed,dtype=np.float64)
mass = np.array(python_mass,dtype=np.float64)
dt = 60 * 60 * 24 * 52

TaskManager.register('get_job_queue')
TaskManager.register('get_result_queue')
server_ip = "192.168.178.203"
server_socket = 12345
taskmanager = TaskManager(address=(server_ip, server_socket), authkey = b'secret')
taskmanager.connect()
job_queue, result_queue = taskmanager.get_job_queue(), taskmanager.get_result_queue()
partsize = (int) (number_of_bodys / 32)
#partsize = 1000

In [25]:
%%timeit
%autoreload
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append("./Pubyc-Cluster-Overhead")

from PubycMaster import single_step
from PubycTaskManager import TaskManager
taskmanager
single_step(job_queue, result_queue, partsize, dt, position, speed, mass)
#10000 1PC 11.3 s ± 273 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#20000 1PC 48 s ± 538 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

#10000 2PCs 9.97 s ± 384 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#20000 2PCs 37.6 s ± 999 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

finished create argument_list with time:  0.011205673217773438 ms
finished jop_queue with time:             138.08703422546387 ms
finished result_queue with time:          12.876033782958984 ms
finished join with time:                  0.24437904357910156 ms

finished create argument_list with time:  0.011205673217773438 ms
finished jop_queue with time:             84.30194854736328 ms
finished result_queue with time:          10.380983352661133 ms
finished join with time:                  0.1633167266845703 ms

finished create argument_list with time:  0.010967254638671875 ms
finished jop_queue with time:             99.2591381072998 ms
finished result_queue with time:          14.152288436889648 ms
finished join with time:                  0.2117156982421875 ms

finished create argument_list with time:  0.012874603271484375 ms
finished jop_queue with time:             99.36094284057617 ms
finished result_queue with time:          9.427309036254883 ms
finished join with time:         

finished jop_queue with time:             89.20025825500488 ms
finished result_queue with time:          6.808280944824219 ms
finished join with time:                  0.1285076141357422 ms

finished create argument_list with time:  0.0133514404296875 ms
finished jop_queue with time:             90.70372581481934 ms
finished result_queue with time:          14.681577682495117 ms
finished join with time:                  0.24461746215820312 ms

finished create argument_list with time:  0.013113021850585938 ms
finished jop_queue with time:             93.55902671813965 ms
finished result_queue with time:          10.466575622558594 ms
finished join with time:                  0.20313262939453125 ms

finished create argument_list with time:  0.01049041748046875 ms
finished jop_queue with time:             92.05341339111328 ms
finished result_queue with time:          18.947124481201172 ms
finished join with time:                  0.28133392333984375 ms

finished create argument_list with 

finished jop_queue with time:             92.6809310913086 ms
finished result_queue with time:          10.042428970336914 ms
finished join with time:                  0.1399517059326172 ms

finished create argument_list with time:  0.01430511474609375 ms
finished jop_queue with time:             99.4424819946289 ms
finished result_queue with time:          15.295982360839844 ms
finished join with time:                  0.18715858459472656 ms

finished create argument_list with time:  0.0095367431640625 ms
finished jop_queue with time:             89.99323844909668 ms
finished result_queue with time:          9.55820083618164 ms
finished join with time:                  0.29754638671875 ms

finished create argument_list with time:  0.016927719116210938 ms
finished jop_queue with time:             84.69510078430176 ms
finished result_queue with time:          11.59977912902832 ms
finished join with time:                  0.1373291015625 ms

finished create argument_list with time:  0.01

In [ ]:
%autoreload
import os
import sys
import pprofile
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append("./Pubyc-Cluster-Overhead")

from PubycMaster import single_step
from PubycTaskManager import TaskManager

profiler = pprofile.Profile()
with profiler:
    single_step(job_queue, result_queue, partsize, dt, position, speed, mass)
profiler.print_stats()